In [16]:
import os
# Supress Warnings
import warnings

from tqdm import tqdm

warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
import pandas as pd
from matplotlib.cm import RdYlGn,jet,RdBu

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

In [17]:
# Calculate NDVI
training_data = pd.read_csv("../data_test/training_data_uhi_index.csv")
print(training_data.columns)
training_data['datetime'] = pd.to_datetime(training_data['datetime'], format='%d-%m-%Y %H:%M')
training_data.describe()

Index(['Longitude', 'Latitude', 'datetime', 'UHI Index'], dtype='object')


,Longitude,Latitude,datetime,UHI Index
count,11229.000000,11229.000000,11229,11229.000000
mean,-73.933927,40.808800,2021-07-24 15:34:29.056906240,1.000001
min,-73.994457,40.758792,2021-07-24 15:01:00,0.956122
25%,-73.955703,40.790905,2021-07-24 15:22:00,0.988577
50%,-73.932968,40.810688,2021-07-24 15:36:00,1.000237
75%,-73.909647,40.824515,2021-07-24 15:48:00,1.011176
max,-73.879458,40.859497,2021-07-24 15:59:00,1.046036
std,0.028253,0.023171,NaN,0.016238


In [18]:
# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])
time_window = "2021-07-23/2021-07-25"
height = 100
width = 100

In [19]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 30}},
)

In [20]:
items = list(search.get_items())
print('This is the number of scenes that touch our region:',len(items))
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

This is the number of scenes that touch our region: 1


In [21]:
resolution = 10  # meters per pixel
scale = resolution / 111320.0 # degrees per pixel for crs=4326

In [22]:
data = stac_load(
    items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    crs="EPSG:4326",  # Latitude-Longitude
    resolution=scale,  # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

df = train_feat = data.to_dataframe().reset_index()
print(df.head())

df['time'] = pd.to_datetime(df['time'])
df['time'] = df['time'].dt.strftime('%d-%m-%Y %H:%M')
display(df)
training_data['datetime'] = pd.to_datetime(training_data['datetime'], format='%d-%m-%Y %H:%M')

training_data.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'}, inplace=True)

    latitude  longitude                    time  spatial_ref  B01  B02   B03  \
0  40.880031 -74.010016 2021-07-24 15:49:11.024         4326  666  639   728   
1  40.880031 -74.009926 2021-07-24 15:49:11.024         4326  666  639   728   
2  40.880031 -74.009837 2021-07-24 15:49:11.024         4326  666  395   579   
3  40.880031 -74.009747 2021-07-24 15:49:11.024         4326  666  562   775   
4  40.880031 -74.009657 2021-07-24 15:49:11.024         4326  710  919  1036   

    B04   B05   B06   B07   B08   B8A   B11   B12  
0   839  1023  2034  2064  1440  2438  1578  1083  
1   839  1023  2034  2064  1440  2438  1578  1083  
2   415   889  1850  2216  2354  2133  1554  1029  
3   688   889  1850  2216  2270  2133  1554  1029  
4  1108  1201  1566  1757  1584  1908  1742  1353  


,latitude,longitude,time,spatial_ref,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12
0,40.880031,-74.010016,24-07-2021 15:49,4326,666,639,728,839,1023,2034,2064,1440,2438,1578,1083
1,40.880031,-74.009926,24-07-2021 15:49,4326,666,639,728,839,1023,2034,2064,1440,2438,1578,1083
2,40.880031,-74.009837,24-07-2021 15:49,4326,666,395,579,415,889,1850,2216,2354,2133,1554,1029
3,40.880031,-74.009747,24-07-2021 15:49,4326,666,562,775,688,889,1850,2216,2270,2133,1554,1029
4,40.880031,-74.009657,24-07-2021 15:49,4326,710,919,1036,1108,1201,1566,1757,1584,1908,1742,1353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2419603,40.750045,-73.860358,24-07-2021 15:49,4326,1175,1412,1362,1602,1858,1940,1908,1588,2050,2202,1989
2419604,40.750045,-73.860268,24-07-2021 15:49,4326,1175,980,1290,1444,1858,1940,1908,1684,2050,2202,1989
2419605,40.750045,-73.860178,24-07-2021 15:49,4326,1182,1202,1326,1416,1530,1557,1903,1842,1797,1909,1721
2419606,40.750045,-73.860088,24-07-2021 15:49,4326,1182,1220,1398,1418,1530,1557,1903,1788,1797,1909,1721


In [23]:
# Function to extract features for a point from a pandas DataFrame
def extract_features_from_dataframe(lat, lon, df):
    # Calculate distances between the point and all rows in df
    df["distance"] = np.sqrt(
        (df["latitude"] - lat) ** 2 + (df["longitude"] - lon) ** 2
    )
    # Find the closest point
    closest_row = df.loc[df["distance"].idxmin()]
    # Return the feature values as a dictionary
    return closest_row.to_dict()

# Iterate over training_data and extract features
features = []
for _, row in df.iterrows():
    lat, lon = row["latitude"], row["longitude"]
    features.append(extract_features_from_dataframe(lat, lon, training_data))

# Combine extracted features with training_data
extracted_features = pd.DataFrame(features)
training_data_with_features = pd.concat([df.reset_index(drop=True), extracted_features.reset_index(drop=True)], axis=1)
training_data_with_features = training_data_with_features.loc[:, ~training_data_with_features.columns.duplicated()]

conversion_factor = 111320  # metros por grau (aproximadamente no equador)
training_data_with_features['distance_meters'] = training_data_with_features['distance'] * conversion_factor

training_data_with_features


,latitude,longitude,time,spatial_ref,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,datetime,UHI Index,distance,distance_meters
0,40.880031,-74.010016,24-07-2021 15:49,4326,666,639,728,839,1023,2034,2064,1440,2438,1578,1083,2021-07-24 15:28:00,1.011056,0.076435,8508.744508
1,40.880031,-74.009926,24-07-2021 15:49,4326,666,639,728,839,1023,2034,2064,1440,2438,1578,1083,2021-07-24 15:28:00,1.011056,0.076351,8499.374146
2,40.880031,-74.009837,24-07-2021 15:49,4326,666,395,579,415,889,1850,2216,2354,2133,1554,1029,2021-07-24 15:28:00,1.011056,0.076267,8490.005221
3,40.880031,-74.009747,24-07-2021 15:49,4326,666,562,775,688,889,1850,2216,2270,2133,1554,1029,2021-07-24 15:28:00,1.011056,0.076183,8480.637738
4,40.880031,-74.009657,24-07-2021 15:49,4326,710,919,1036,1108,1201,1566,1757,1584,1908,1742,1353,2021-07-24 15:28:00,1.011056,0.076098,8471.271700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2419603,40.750045,-73.860358,24-07-2021 15:49,4326,1175,1412,1362,1602,1858,1940,1908,1588,2050,2202,1989,2021-07-24 15:50:00,0.994828,0.058837,6549.776738
2419604,40.750045,-73.860268,24-07-2021 15:49,4326,1175,980,1290,1444,1858,1940,1908,1684,2050,2202,1989,2021-07-24 15:50:00,0.994828,0.058867,6553.039279
2419605,40.750045,-73.860178,24-07-2021 15:49,4326,1182,1202,1326,1416,1530,1557,1903,1842,1797,1909,1721,2021-07-24 15:50:00,0.994828,0.058896,6556.315449
2419606,40.750045,-73.860088,24-07-2021 15:49,4326,1182,1220,1398,1418,1530,1557,1903,1788,1797,1909,1721,2021-07-24 15:50:00,0.994828,0.058926,6559.605228


In [24]:
# Exemplo: Criação de NDVI e SAVI
training_data_with_features['NDVI'] = (training_data_with_features['B08'] - training_data_with_features['B04']) / (training_data_with_features['B08'] + training_data_with_features['B04'] + 1e-6)
L = 0.5
training_data_with_features['SAVI'] = ((training_data_with_features['B08'] - training_data_with_features['B04']) * (1 + L)) / (training_data_with_features['B08'] + training_data_with_features['B04'] + L + 1e-6)

# Criação de NDBI
training_data_with_features['NDBI'] = (training_data_with_features['B11'] - training_data_with_features['B08']) / (training_data_with_features['B11'] + training_data_with_features['B08'] + 1e-6)

# Criação de MNDWI
training_data_with_features['MNDWI'] = (training_data_with_features['B03'] - training_data_with_features['B11']) / (training_data_with_features['B03'] + training_data_with_features['B11'] + 1e-6)

# Criação de EVI
training_data_with_features['EVI'] = 2.5 * (training_data_with_features['B08'] - training_data_with_features['B04']) / (training_data_with_features['B08'] + 6 * training_data_with_features['B04'] - 7.5 * training_data_with_features['B02'] + 1)

In [26]:
print(training_data_with_features.shape)
training_data_with_features_filtered_100 = training_data_with_features[training_data_with_features['distance_meters'] < 100]
training_data_with_features_filtered_200 = training_data_with_features[training_data_with_features['distance_meters'] < 200]
training_data_with_features_filtered_300 = training_data_with_features[training_data_with_features['distance_meters'] < 300]

print("Data shape less than 100m:", training_data_with_features_filtered_100.shape)
print("Data shape less than 200m:", training_data_with_features_filtered_200.shape)
print("Data shape less than 300m:", training_data_with_features_filtered_300.shape)

(2419608, 24)
Data shape less than 100m: (211236, 24)
Data shape less than 200m: (385730, 24)
Data shape less than 300m: (507229, 24)


In [28]:
import pandas as pd
import numpy as np


def remove_outliers_iqr(df, factor=3):
    print(df)
    df.reset_index(drop=True)
    df_clean = df.copy()
    # Get list of numeric columns
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()

    for col in numeric_cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR

        # Keep rows within the bounds
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# Example usage:
# Assuming df is your DataFrame
df_clean_factor3 = remove_outliers_iqr(training_data_with_features_filtered_100, factor=3)
df_clean_factor4 = remove_outliers_iqr(training_data_with_features_filtered_100, factor=4)
df_clean_factor5 = remove_outliers_iqr(training_data_with_features_filtered_100, factor=5)
df_clean_factor2 = remove_outliers_iqr(training_data_with_features_filtered_100, factor=2)
print("Data shape before outlier removal:", training_data_with_features_filtered_100.shape)
print("Data shape after outlier removal:", df_clean_factor3.shape)


          latitude  longitude              time  spatial_ref  B01  B02   B03  \
366814   40.860358 -73.932312  24-07-2021 15:49         4326  259  350   475   
366815   40.860358 -73.932222  24-07-2021 15:49         4326  259  273   445   
366816   40.860358 -73.932133  24-07-2021 15:49         4326  259  273   445   
366817   40.860358 -73.932043  24-07-2021 15:49         4326  259  229   403   
366818   40.860358 -73.931953  24-07-2021 15:49         4326  259  239   335   
...            ...        ...               ...          ...  ...  ...   ...   
2271447  40.757950 -73.959890  24-07-2021 15:49         4326  889  602   760   
2271448  40.757950 -73.959801  24-07-2021 15:49         4326  889  508   824   
2271449  40.757950 -73.959711  24-07-2021 15:49         4326  889  508   824   
2271450  40.757950 -73.959621  24-07-2021 15:49         4326  889  618   821   
2271451  40.757950 -73.959531  24-07-2021 15:49         4326  956  842  1023   

         B04   B05   B06  ...   B12    

In [29]:

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import VarianceThreshold, SelectFromModel, SelectKBest, f_regression
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler

# ------------------------------
# 1. Carregar Dados e Pré-processamento / Load Data and Preprocessing
# ------------------------------
# Ajuste o caminho do seu dataset conforme necessário
data = df_clean_factor3.copy()

# Remover colunas irrelevantes / Remove irrelevant columns
cols_to_drop = ["latitude", "longitude", "datetime", "distance", "distance_meters", "time", "spatial_ref", "EVI"]
data = data.drop(columns=cols_to_drop, errors="ignore")

# Definir a variável target e as features / Define target variable and features
target = "UHI Index"
X = data.drop(target, axis=1)
y = data[target]

# ------------------------------
# 2. Construção do Pipeline / Pipeline Construction
# ------------------------------
pipeline = Pipeline([
    # Seleção de features baseada em modelo (ExtraTreesRegressor)
    ("select_from_model", SelectFromModel(
        estimator=ExtraTreesRegressor(n_estimators=300, random_state=42),
        threshold="0.99*mean"
    )),
    # Escalonamento robusto dos dados
    ("scaler", RobustScaler()),
    # Modelo: ExtraTreesRegressor com hiperparâmetros otimizados
    ("model", ExtraTreesRegressor(random_state=42, n_estimators=600, max_depth=100, min_samples_split=2))
])

# ------------------------------
# 3. Validação Cruzada / Cross Validation
# ------------------------------
# Utilizando 5 folds
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(
    pipeline, X, y, cv=cv,
    scoring={"r2": "r2", "mape": "neg_mean_absolute_percentage_error"},
    return_train_score=False
)

# Cálculo das métricas médias e desvio padrão / Calculate average metrics and standard deviations
mean_r2 = cv_results["test_r2"].mean()
std_r2 = cv_results["test_r2"].std()
# Converter MAPE para valor positivo e percentual
mean_mape = -cv_results["test_mape"].mean() * 100
std_mape = cv_results["test_mape"].std() * 100

print("Validação Cruzada R²: {:.4f} ± {:.4f}".format(mean_r2, std_r2))
print("Validação Cruzada MAPE: {:.2f}% ± {:.2f}%".format(mean_mape, std_mape))

# ------------------------------
# 4. Treinamento Final com Todos os Dados / Final Training with All Data
# ------------------------------
pipeline.fit(X, y)


Validação Cruzada R²: 0.9539 ± 0.0013
Validação Cruzada MAPE: 0.16% ± 0.00%


Pipeline(steps=[('select_from_model',
                 SelectFromModel(estimator=ExtraTreesRegressor(n_estimators=300,
                                                               random_state=42),
                                 threshold='0.99*mean')),
                ('scaler', RobustScaler()),
                ('model',
                 ExtraTreesRegressor(max_depth=100, n_estimators=600,
                                     random_state=42))])

In [ ]:
sub_temp = pd.read_csv('../data/Submission_template.csv')
sub_temp.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}, inplace=True)
features = []
for _, row in sub_temp.iterrows():
    lat, lon = row["latitude"], row["longitude"]
    features.append(extract_features_from_dataframe(lat, lon, train_feat))

val_features = pd.DataFrame(features)
val_data_with_features = pd.concat([sub_temp.reset_index(drop=True), val_features.reset_index(drop=True)], axis=1)
val_data_with_features = val_data_with_features.loc[:, ~val_data_with_features.columns.duplicated()]
val_data_with_features

In [ ]:
# Exemplo: Criação de NDVI e SAVI
val_data_with_features['NDVI'] = (val_data_with_features['B08'] - val_data_with_features['B04']) / (val_data_with_features['B08'] + val_data_with_features['B04'] + 1e-6)
L = 0.5
val_data_with_features['SAVI'] = ((val_data_with_features['B08'] - val_data_with_features['B04']) * (1 + L)) / (val_data_with_features['B08'] + val_data_with_features['B04'] + L + 1e-6)

# Criação de NDBI
val_data_with_features['NDBI'] = (val_data_with_features['B11'] - val_data_with_features['B08']) / (val_data_with_features['B11'] + val_data_with_features['B08'] + 1e-6)

# Criação de MNDWI
val_data_with_features['MNDWI'] = (val_data_with_features['B03'] - val_data_with_features['B11']) / (val_data_with_features['B03'] + val_data_with_features['B11'] + 1e-6)

# Criação de EVI
val_data_with_features['EVI'] = 2.5 * (val_data_with_features['B08'] - val_data_with_features['B04']) / (val_data_with_features['B08'] + 6 * val_data_with_features['B04'] - 7.5 * val_data_with_features['B02'] + 1)

In [ ]:
# val_data_with_features = val_data_with_features[['longitude', 'latitude', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06','B07', 'B08', 'B8A', 'B11', 'B12']]
copy_val_data = val_data_with_features[['B01', 'B02', 'B03', 'B04', 'B05', 'B06','B07', 'B08', 'B8A', 'B11', 'B12', 'NDVI', 'SAVI', 'NDBI', 'MNDWI']].copy()
pred_vals = pipeline.predict(copy_val_data)

In [ ]:
pred_vals

In [ ]:
data_to_send = pd.DataFrame()
data_to_send['UHI Index'] = pred_vals
data_to_send['Latitude'] = val_data_with_features['latitude']
data_to_send['Longitude'] = val_data_with_features['longitude']

data_to_send = data_to_send[['Longitude', 'Latitude', 'UHI Index']]
data_to_send.to_csv('../outputs/v4_predicted_values.csv', index=False)